In [1]:
import asf_search as asf
from shapely import Polygon, box
import numpy as np
import pyproj
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.merge
import os
from rasterio.transform import from_origin
import geopandas as gpd
from dem_stitcher import stitch_dem
import zipfile
from urllib.request import urlretrieve
import geopandas as gpd
from shapely import Polygon
import requests


# Functions

In [16]:
def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def adjust_scene_poly_at_extreme_lat(bbox, src_crs, ref_crs, delta=0.1):
    """
    Adjust the bounding box around a scene in src_crs (4326) due to warping at high
    Latitudes. For example, the min and max boudning values for an antarctic scene in
    4326 may not actually be the true min and max due to distortions at high latitudes. 

    Parameters:
    - bbox: Tuple of four coordinates (x_min, y_min, x_max, y_max).
    - src_crs: Source EPSG. e.g. 4326
    - ref_crs: reference crs to create the true bbox. i.e. 3031 in southern 
                hemisphere and 3995 in northern (polar stereographic)
    - delta: distance between generation points along the bounding box sides in
            src_crs. e.g. 0.1 degrees in lat/lon 

    Returns:
    - A polygon bounding box expanded to the true min max
    """
    x_min, y_min, x_max, y_max = bbox
    # Generate points along the top side
    top_side = [(x, y_max) for x in list(np.arange(x_min, x_max, delta)) + [x_max]]    
    # Generate points along the right side
    right_side = [(x_max, y) for y in list(np.arange(y_max, y_min, -delta)) + [y_min]]
    # Generate points along the bottom side
    bottom_side = [(x, y_min) for x in list(np.arange(x_max, x_min, -delta)) + [x_min]]
    # Generate points along the left side
    left_side = [(x_min, y) for y in list(np.arange(y_min, y_max, delta)) + [y_max]]
    # Combine all sides' points
    all_points = top_side + right_side + bottom_side + left_side
    # convert to a polygon 
    polygon = Polygon(all_points)
    # convert polygon to desired crs and get bounds in those coordinates
    trans_bounds = transform_polygon(src_crs, ref_crs, polygon).bounds
    trans_poly = Polygon(
        [(trans_bounds[0], trans_bounds[1]), 
         (trans_bounds[2], trans_bounds[1]), 
         (trans_bounds[2], trans_bounds[3]), 
         (trans_bounds[0], trans_bounds[3])]
        )
    corrected_poly = transform_polygon(ref_crs, src_crs, trans_poly)
    return corrected_poly

def reproject_raster(in_path: str, out_path: str, crs: int):
    """Reproject a raster to the desired crs

    Args:
        in_path (str): path to src raster
        out_path (str): save path of reproj raster
        crs (int): crs e.g. 3031

    Returns:
        str: save path of reproj raster
    """
    # reproject raster to project crs
    with rasterio.open(in_path) as src:
        src_crs = src.crs
        transform, width, height = calculate_default_transform(
            src_crs, crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()

        # get crs proj 
        crs = pyproj.CRS(f"EPSG:{crs}")

        kwargs.update({
            'crs': crs,
            'transform': transform,
            'width': width,
            'height': height})

        with rasterio.open(out_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=crs,
                    resampling=Resampling.bilinear)
    return out_path

def find_files(folder, contains):
    paths = []
    for root, dirs, files in os.walk(folder):
        for name in files:
            if contains in name:
                filename = os.path.join(root,name)
                paths.append(filename)
    return paths

def get_REMA_index_file(save_folder):
    rema_index_url = 'https://data.pgc.umn.edu/elev/dem/setsm/REMA/indexes/REMA_Mosaic_Index_latest_gdb.zip'
    filename = 'REMA_Mosaic_Index_latest_gdb.zip'
    # download and store locally
    zip_save_path = os.path.join(save_folder, filename)
    urlretrieve(rema_index_url, zip_save_path)
    #unzip 
    with zipfile.ZipFile(zip_save_path, 'r') as zip_ref:
        zip_ref.extractall(save_folder)
        files=zip_ref.infolist()
        rema_index_file = '/'.join(files[0].filename.split('/')[0:-1])
    rema_index_path = os.path.join(save_folder, rema_index_file)
    os.remove(zip_save_path)
    return rema_index_path

def get_REMA_tiles(s3_url_list, resolution, save_folder):

    valid_res = [2, 10, 32, 100, 500, 1000]
    assert resolution in valid_res, f"resolution must be in {valid_res}"

    # format for request, all metres except 1km
    resolution = f'{resolution}m' if resolution != 1000 else '1km'

    # download individual dems
    dem_paths = []
    for i, s3_file_url in enumerate(s3_url_list):
        # all urls are for 10m, set to other baws on resololution
        s3_file_url = s3_file_url.replace('10m',f'{resolution}')
        # get the raw json url
        json_url = f'https://{s3_file_url.split("external/")[-1]}'
        # Make a GET request to fetch the raw JSON content
        response = requests.get(json_url)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse JSON content into a Python dictionary
            data = response.json()
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
        
        dem_url = json_url.replace('.json', '_dem.tif')
        local_path = os.path.join(save_folder, dem_url.split('amazonaws.com')[1][1:])
        local_folder = '/'.join(local_path.split('/')[0:-1])
        # check if the dem.tif already exists
        dem_path = find_files(local_folder, 'dem.tif')
        if len(dem_path) > 0:
            print(f'{dem_path[0]} already exists, skipping download')
            dem_paths.append(dem_path[0])
            continue
        os.makedirs(local_folder, exist_ok=True)
        print(f'downloading {i+1} of {len(s3_url_list)}: src: {dem_url} dst: {local_path}')
        urlretrieve(dem_url, local_path)
        dem_paths.append(dem_url)

    return dem_paths


def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def check_s1_bounds_cross_antimeridian(bounds, max_scene_width=20):
    """Check if the s1 bounds cross the antimeridian. We set a max
    scene width of 10. if the scene is greater than this either side
    of the dateline, we assume it crosses the dateline as the
    alternate scenario is a bounds with a very large width.

    e.g. [-178.031982, -71.618958, 178.577438, -68.765755]

    Args:
        bounds (list or tuple): Bounds in 4326
        max_scene_width (int, optional): maxumum width of bounds in 
        lon degrees. Defaults to 20.

    Returns:
        bool: True if crosses the antimeridian
    """

    min_x = -180 + max_scene_width # -160
    max_x = 180 - max_scene_width # 160
    if (bounds[0] < min_x) and (bounds[0] > -180):
        if bounds[2] > max_x and bounds[2] < 180:
            return True
    
def split_am_crossing(scene_polygon, lat_buff=0.2):
    """split the polygon into bounds on the left and
    right of the antimeridian.

    Args:
        scene_polygon (shapely.polygon): polygon of the scene from asf
        lat_buff (float): A lattitude degrees buffer to add/subtract from
                            max and min lattitudes 

    Returns:
    list(tuple) : a list containing two sets of bounds for the left
                    and right of the antimeridian
    """
    max_negative_x = max([point[0] for point in scene_polygon.exterior.coords if point[0] < 0])
    min_positive_x = min([point[0] for point in scene_polygon.exterior.coords if point[0] > 0])
    min_y = min([point[1] for point in scene_polygon.exterior.coords]) - lat_buff
    max_y = max([point[1] for point in scene_polygon.exterior.coords]) + lat_buff
    min_y = -90 if min_y < -90 else min_y
    max_y = 90 if max_y > 90 else max_y
    bounds_left = (-180, min_y, max_negative_x, max_y)
    bounds_right = (min_positive_x, min_y, 180, max_y)
    return [tuple(bounds_left), tuple(bounds_right)]

Search for sentinel 1 scenes to get some geographical bounds

In [ ]:
prod = 'SLC'
mode = 'IW'

scene_list = [
    # antimeridian scene
    #'S1A_IW_SLC__1SDV_20191028T131928_20191028T131947_029659_0360CA_A1A0',

    # high latittude scene - Ross Island 
    'S1A_IW_SLC__1SSH_20211220T124745_20211220T124815_041092_04E1C2_0475'
]

# search the scene list with the specified product type
results = asf.granule_search(scene_list, asf.ASFSearchOptions(processingLevel=prod))
print(f'number of results found : {len(results)}')

Search for bounds

In [4]:
# prod= 'GRD'
# mode = 'EW'
# wkt = "POINT (-24.984 -73.620)" 
# start='2024-02-29T11:59:59Z'
# end='2024-03-31T11:59:59Z'

# # search the scene list with the specified product type
# results = asf.search(
#     platform=[asf.PLATFORM.SENTINEL1],
#     intersectsWith= wkt,
#     maxResults=1000,
#     beamMode=mode,
#     #processingLevel=prod,
#     start=start,
#     end=end
#     )
# print(f'number of results found : {len(results)}')
# for r in results:
#     print(r.properties['sceneName'])

In [ ]:
geom = results[0].geometry
scene_polygon = Polygon(geom['coordinates'][0])
bounds = scene_polygon.bounds
bounds

In [ ]:
scene_polygon

In [7]:
scene_polygon_buf = scene_polygon.buffer(0.1)
bounds_buf = scene_polygon_buf.bounds

## Compernicus 30m

In [ ]:
antimeridian_crossing = check_s1_bounds_cross_antimeridian(bounds, max_scene_width=8)
if antimeridian_crossing:
    trg_crs = 3031
    print('crossing AM')
    # split into bounds either side of the AM
    bounds_left, bounds_right = split_am_crossing(scene_polygon, lat_buff=0.2)
    # get left dem
    bounds_left = adjust_scene_poly_at_extreme_lat(bounds_left, 4326, 3031).bounds
    dem_data, dem_meta = stitch_dem(bounds_left,
        dem_name='glo_30',
        dst_ellipsoidal_height=True,
        dst_area_or_point='Point',
        merge_nodata_value=0,
        fill_to_bounds=True,
    )
    with rasterio.open('tmp_left_dem.tif', 'w', **dem_meta) as ds:
        ds.write(dem_data,1)
        ds.update_tags(AREA_OR_POINT='Point')
    # reproject left dem to target crs
    #   - warning dem is at AM, crs to handle this is required
    #   - raise error if 4326?
    reproject_raster('tmp_left_dem.tif',f'tmp_left_dem_{trg_crs}.tif',trg_crs)
    # get right dem
    bounds_right = adjust_scene_poly_at_extreme_lat(bounds_right, 4326, 3031).bounds
    dem_data, dem_meta = stitch_dem(bounds_right,
        dem_name='glo_30',
        dst_ellipsoidal_height=True,
        dst_area_or_point='Point',
        merge_nodata_value=0,
        fill_to_bounds=True,
    )
    # check target crs
    with rasterio.open('tmp_right_dem.tif', 'w', **dem_meta) as ds:
        ds.write(dem_data,1)
        ds.update_tags(AREA_OR_POINT='Point')
    # reproject right dem to target crs
    reproject_raster('tmp_right_dem.tif',f'tmp_right_dem_{trg_crs}.tif',trg_crs)
    # merge dems
    rasterio.merge.merge(
        sources=[f'tmp_left_dem_{trg_crs}.tif',f'tmp_right_dem_{trg_crs}.tif'],
        dst_path='dem.tif')
    # merge dems

else:
    print('not crossing AM')
    new_poly = adjust_scene_poly_at_extreme_lat(bounds, 4326, 3031, delta=0.01)
    new_bounds = new_poly.bounds
    dem_data, dem_meta = stitch_dem(new_bounds,
        dem_name='glo_30',
        dst_ellipsoidal_height=True,
        dst_area_or_point='Point',
        merge_nodata_value=0,
        fill_to_bounds=True,
    )
    with rasterio.open('cop30m_dem.tif', 'w', **dem_meta) as ds:
        ds.write(dem_data,1)
        ds.update_tags(AREA_OR_POINT='Point')

# REMA DEM

In [8]:
# convert to 3031
scene_poly_3031 = transform_polygon(4326, 3031, scene_polygon)
# buffer by 10km each size
buf = 10_000
scene_poly_3031_buf = scene_poly_3031.buffer(buf)

In [9]:
resolution = 32 # must be one of [2, 10, 32, 100, 500, 1000]
dem_filename = f'REMA{resolution}_dem.tif'

In [ ]:
os.makedirs('data', exist_ok=True)
print('Downloading REMA index file')
rema_index_path = get_REMA_index_file(save_folder='data')

In [ ]:
# load into gpdf
rema_index_df = gpd.read_file(rema_index_path)
# find the intersecting tiles
intersecting_rema_files = rema_index_df[
    rema_index_df.geometry.intersects(scene_poly_3031_buf)]
s3_url_list = intersecting_rema_files['s3url'].to_list()
print(f'{len(s3_url_list)} intersecting tiles found')
dem_paths = get_REMA_tiles(s3_url_list[0:], resolution, save_folder='data')

In [ ]:
# combine DEMS
# for some reason this may hand on first run, you may need to restart and run again
print('combining DEMS')
rasterio.merge.merge(
        datasets=dem_paths,
        dst_path=dem_filename
        )


# reporject to desired crs
# if str(crs) != '3031':
#     reproj_dem_path = os.path.join('data',dem_filename)
#     print(f'reprojecting DEM to {crs}')
#     reproject_raster(merge_dem_path, reproj_dem_path, crs)
#     os.remove(merge_dem_path)
#     return reproj_dem_path
